# Transformer (DTU DATA)

## Needed for Google CoLab

In [3]:
# To run on Colab, you need to run this code
!pip install torch torchvision torchaudio pandas numpy scikit-learn

In [5]:
from google.colab import files

uploaded = files.upload()

Saving AveragePower.xlsx to AveragePower.xlsx


## Importing Excel

In [7]:
import pandas as pd
import numpy as np

# Read the dataset
#df = pd.read_excel('../Dataset/AveragePower.xlsx')
df = pd.read_excel('AveragePower.xlsx')


# Split the data into 2022 and 2023 based on the 'Year' column
df_2022 = df[df['Year'] == 2022].reset_index(drop=True)
df_2023 = df[df['Year'] == 2023].reset_index(drop=True)

# Split the 2022 and 2023 data by 'DayType' (Weekday, Weekend)
df_2022_weekday = df_2022[df_2022['DayType'] == 'Weekday'].reset_index(drop=True)
df_2022_weekend = df_2022[df_2022['DayType'] == 'Weekend'].reset_index(drop=True)
df_2023_weekday = df_2023[df_2023['DayType'] == 'Weekday'].reset_index(drop=True)
df_2023_weekend = df_2023[df_2023['DayType'] == 'Weekend'].reset_index(drop=True)

print("\n2022 Data");
print(df_2022_weekday.head())
print(df_2022_weekend.head())

print("\n2023 Data")
print(df_2023_weekday.head())
print(df_2023_weekend.head())


2022 Data
   Year  Month  Hour  DayType     power
0  2022      1     0  Weekday  0.943937
1  2022      1     1  Weekday  0.955728
2  2022      1     2  Weekday  0.772173
3  2022      1     3  Weekday  0.530580
4  2022      1     4  Weekday  0.333006
   Year  Month  Hour  DayType     power
0  2022      1     0  Weekend  0.680075
1  2022      1     1  Weekend  0.739797
2  2022      1     2  Weekend  0.605256
3  2022      1     3  Weekend  0.440174
4  2022      1     4  Weekend  0.310420

2023 Data
   Year  Month  Hour  DayType     power
0  2023      1     0  Weekday  0.881067
1  2023      1     1  Weekday  1.288525
2  2023      1     2  Weekday  1.312552
3  2023      1     3  Weekday  1.074686
4  2023      1     4  Weekday  0.573104
   Year  Month  Hour  DayType     power
0  2023      1     0  Weekend  0.805268
1  2023      1     1  Weekend  1.142235
2  2023      1     2  Weekend  1.186799
3  2023      1     3  Weekend  1.097171
4  2023      1     4  Weekend  0.780017


## Preprocess Data

In [8]:
from sklearn.preprocessing import StandardScaler

def preprocess_data(df):
  # Encode 'Hour' using sine and cosine
  df['Hour_sin'] = np.sin(2 * np.pi * df['Hour']/24)
  df['Hour_cos'] = np.cos(2 * np.pi * df['Hour']/24)

  # Encode 'month' using sine and cosine
  df['Month_sin'] = np.sin(2 * np.pi * df['Month']/12)
  df['Month_cos'] = np.cos(2 * np.pi * df['Month']/12)

  # Scale 'Power' using StandardScaler
  scaler = StandardScaler()
  df['Power'] = scaler.fit_transform(df[['power']])

  return df, scaler

# Proprocess Weekday data and Weekend data for 2022
df_2022_weekday, scaler_weekday = preprocess_data(df_2022_weekday)
df_2022_weekend, scaler_weekend = preprocess_data(df_2022_weekend)

# Proprocess Weekday data and Weekend data for 2023
df_2023_weekday, _ = preprocess_data(df_2023_weekday)
df_2023_weekend, _ = preprocess_data(df_2023_weekend)


## Squence Creation

In [9]:
import torch
# Function to create sequences for the model
def create_sequences(df, window_size):
    sequences = []
    labels = []

    # Ensure that the data is numeric before processing
    # Convert all columns to numeric
    df = df.apply(pd.to_numeric, errors='coerce')

    for i in range(len(df) - window_size):
        # Drop power column for features
        seq = df.iloc[i:i + window_size].drop(columns=['Power']).values
        # Target is the next hour's power value
        label = df.iloc[i + window_size]['Power']
        sequences.append(seq)
        labels.append(label)

    return np.array(sequences), np.array(labels)

# Create sequences for a month (24 hours)
sequence_length = 24

# Create sequences for Weekday and Weekend data
X_train_weekday, y_train_weekday = create_sequences(
    df_2022_weekday, sequence_length)
X_train_weekday = X_train_weekday.astype(np.float32)  # Ensure float32
X_train_weekday = torch.tensor(X_train_weekday, dtype=torch.float32)
X_train_weekday = np.nan_to_num(X_train_weekday, nan=0.0).astype(np.float32)
X_train_weekend, y_train_weekend = create_sequences(df_2022_weekend, sequence_length)


X_test_weekday, y_test_weekday = create_sequences(df_2023_weekday, sequence_length)
X_test_weekend, y_test_weekend = create_sequences(df_2023_weekend, sequence_length)

# Transformer Model

In [14]:
import torch
import torch.nn as nn

class PowerPredictor(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, output_dim):
        super(PowerPredictor, self).__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.nhead = nhead
        self.num_layers = num_layers
        self.output_dim = output_dim

        # Ensure input features match d_model
        # Map input features to d_model
        self.fc_input = nn.Linear(input_dim, d_model)

        # Transformer model
        self.transformer = nn.Transformer(
            d_model=d_model, nhead=nhead, num_encoder_layers=num_layers)

        # Fully connected layer for output
        self.fc = nn.Linear(d_model, output_dim)

    def forward(self, x):
        # Apply fully connected layer to match input to d_model
        # (batch_size, seq_len, input_dim) -> (batch_size, seq_len, d_model)
        x = self.fc_input(x)

        # Transformer expects input shape (seq_len, batch_size, feature_dim)
        # (batch_size, seq_len, feature_dim) -> (seq_len, batch_size, feature_dim)
        x = x.permute(1, 0, 2)
        transformer_output = self.transformer(x, x)
        # Use the last output to predict next value
        out = self.fc(transformer_output[-1])
        return out

# Train the Model

In [15]:
import numpy as np
import torch
import torch.nn as nn

# Hyperparameters
input_dim = X_train_weekday.shape[2]  # Number of features (e.g., Hour, Month)
d_model = 128  # Transformer model dimension
nhead = 4  # Number of heads in MultiheadAttention
num_layers = 2  # Number of Transformer layers
output_dim = 1  # Single output value

# Create model
model_weekend = PowerPredictor(
    input_dim, d_model, nhead, num_layers, output_dim)


# Check data types and handle missing values
assert X_train_weekday.dtype in [
    np.float32, np.float64], "X_train_weekday must be float32 or float64"
assert y_train_weekday.dtype in [
    np.float32, np.float64], "y_train_weekday must be float32 or float64"

X_train_weekday = np.nan_to_num(X_train_weekday, nan=0.0).astype(np.float32)
y_train_weekday = y_train_weekday.astype(np.float32)

# Convert data to PyTorch tensors
X_train_weekday_tensor = torch.tensor(X_train_weekday, dtype=torch.float32)
y_train_weekday_tensor = torch.tensor(y_train_weekday, dtype=torch.float32)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model_weekend.parameters(), lr=0.001)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model_weekend.train()

    # Forward pass
    output = model_weekend(X_train_weekday_tensor)
    loss = criterion(output.squeeze(), y_train_weekday_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:  # Log every 10 epochs
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch [10/50], Loss: 1.2158
Epoch [20/50], Loss: 1.0185
Epoch [30/50], Loss: 1.0045
Epoch [40/50], Loss: 1.0364
Epoch [50/50], Loss: 0.9996


# Test the Model

In [16]:
# Convert 2023 Weekend test data to PyTorch tensor
X_test_weekend_tensor = torch.tensor(X_test_weekend, dtype=torch.float32)

# Set the model to evaluation mode
model_weekend.eval()

# Make predictions for 2023 Weekend data
with torch.no_grad():
    y_pred_weekend = model_weekend(X_test_weekend_tensor).squeeze()

# Inverse scale the predictions and actual values to the original scale
y_pred_weekend_original = scaler_weekend.inverse_transform(
    y_pred_weekend.numpy().reshape(-1, 1)).flatten()
y_test_weekend_original = scaler_weekend.inverse_transform(
    y_test_weekend.reshape(-1, 1)).flatten()

## Evaluate the Weekend model

In [22]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import numpy as np

print(y_pred_weekend_original)
print(y_pred_weekend_original.shape)


print(np.isnan(y_test_weekend_original).sum())
print(np.isnan(y_pred_weekend_original).sum())

y_test_weekend_original = np.nan_to_num(y_test_weekend_original, nan=np.nanmean(y_test_weekend_original))
y_pred_weekend_original = np.nan_to_num(y_pred_weekend_original, nan=np.nanmean(y_pred_weekend_original))

mask = ~np.isnan(y_test_weekend_original) & ~np.isnan(y_pred_weekend_original)
y_test_weekend_original = y_test_weekend_original[mask]
y_pred_weekend_original = y_pred_weekend_original[mask]


mae_weekend = mean_absolute_error(y_test_weekend_original, y_pred_weekend_original)
mse_weekend = mean_squared_error(y_test_weekend_original, y_pred_weekend_original)
r2_weekend = r2_score(y_test_weekend_original, y_pred_weekend_original)

print(f'MAE for 2023 Weekend data: {mae_weekend:.2f}')
print(f'MSE for 2023 Weekend data: {mse_weekend:.2f}')
print(f'R2 for 2023 Weekend data: {r2_weekend:.2f}')

[]
(0,)
0
0


<ipython-input-22-e57b25a327bd>:12: RuntimeWarning: Mean of empty slice
  y_test_weekend_original = np.nan_to_num(y_test_weekend_original, nan=np.nanmean(y_test_weekend_original))
<ipython-input-22-e57b25a327bd>:13: RuntimeWarning: Mean of empty slice
  y_pred_weekend_original = np.nan_to_num(y_pred_weekend_original, nan=np.nanmean(y_pred_weekend_original))


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.